# Recomanacions Anime


Important dades 

In [ ]:
import pandas as pd

ratings = pd.read_csv('/Users/alexmiliandieguez/code/jupyter/data/anime/rating.csv', sep=',', header = 0)

m_cols = ['anime_id', 'name']
anime = pd.read_csv('/Users/alexmiliandieguez/code/jupyter/data/anime/anime.csv', sep=',', names=m_cols, usecols=range(2), header = 0)

print(ratings.head())
print(ratings.describe())

Neteja de ratings negatius o sense ratings

In [3]:
ratings['rating'] = pd.to_numeric(ratings['rating'], errors='coerce')
ratings = ratings.dropna(subset=['rating'])
ratings = ratings[ratings['rating'] >= 0]
print(ratings.head())
print(ratings.describe())


Contador de quantes vegades s'han valorat els anime

In [ ]:
counts = ratings['anime_id'].value_counts()
users = ratings['user_id'].value_counts()

print("Ratings per anime")
print(counts.describe())
print("\n")
print(f"Ratings per user")
print(users.describe())

Gràfics

In [ ]:
import matplotlib.pyplot as plt

plt.hist(users, bins=50)
plt.title('Distribució de nombre de valoracions per usuari')
plt.xlabel('Nombre de valoracions per usuari')
plt.ylabel('Nombre d\'usuaris')
plt.show()

plt.hist(counts, bins=50)
plt.title('Distribució de nombre de valoracions per anime')
plt.xlabel('Nombre de valoracions per anime')
plt.ylabel('Nombre d\'animes')
plt.show()

print(f"Usuaris percentil 99% : {users.quantile(0.99)}")


Neteja de dades

In [ ]:
N_min = 100
counts = ratings['anime_id'].value_counts()
anime_significatius = counts[counts >= N_min].index
ratings_filtrat = ratings[ratings['anime_id'].isin(anime_significatius)]

U_min = 10
U_max = 517
users = ratings_filtrat['user_id'].value_counts()
usuaris_actius = users[users >= U_min].index
usuaris_actius = usuaris_actius[users[usuaris_actius] <= U_max]
ratings_filtrat = ratings_filtrat[ratings_filtrat['user_id'].isin(usuaris_actius)]

counts = ratings_filtrat['anime_id'].value_counts()
users = ratings_filtrat['user_id'].value_counts()

print("Ratings per anime")
print(counts.describe())
print("\n")
print(f"Ratings per user")
print(users.describe())


print(f"\nAnimes unitat: {ratings_filtrat['anime_id'].nunique()}")
print(f"Usuaris unitat: {ratings_filtrat['user_id'].nunique()}")
print(f"Total de files: {len(ratings_filtrat)}")

Gràfcis

In [ ]:
plt.hist(users, bins=50)
plt.title('Distribució de nombre de valoracions per usuari')
plt.xlabel('Nombre de valoracions per usuari')
plt.ylabel('Nombre d\'usuaris')
plt.show()

plt.hist(counts, bins=50)
plt.title('Distribució de nombre de valoracions per anime')
plt.xlabel('Nombre de valoracions per anime')
plt.ylabel('Nombre d\'animes')
plt.show()


Pivotar taula

In [ ]:
userRatings = ratings_filtrat.pivot_table(index=['user_id'],columns=['anime_id'],values='rating')
userRatings.head()

Matriu de correlació

In [ ]:
corrMatrix = userRatings.corr(method='pearson', min_periods=500)

In [ ]:
print(corrMatrix.describe())

Afegim usuari

In [ ]:
myRatings = pd.Series({11061: 10, 2476: 1}, name=0)
userRatings = pd.concat([userRatings, myRatings.to_frame().T])
print(myRatings)

Canviem id_anime a name

In [ ]:
name_map = anime.set_index('anime_id')['name'].to_dict()
userRatings = userRatings.rename(columns=name_map)
corrMatrix_names = corrMatrix.rename(index=name_map, columns=name_map)
myRatings = userRatings.loc[0].dropna()
print(myRatings)

In [ ]:
simCandidates = pd.Series(dtype='float64')

for anime, rating in myRatings.items():
    sims = corrMatrix_names[anime].dropna()
    sims = sims * rating
    simCandidates = pd.concat([simCandidates, sims])

simCandidates = simCandidates.groupby(simCandidates.index).sum()

filteredSims = simCandidates.drop(myRatings.index)

filteredSims = filteredSims.sort_values(ascending=False)

print(filteredSims.head(10))
